In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("MainDataSet1D.csv").tail(100)

In [127]:
df.columns


Index(['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_SuperTrend',
       'Day_Candle', 'Day_CandleBody', 'Day_5EMA', 'Day_15EMA', 'Day_20EMA',
       'Day_50EMA', 'Day_200EMA', 'Day_RSI', 'Day_Date', 'Day_SwingMin',
       'Day_SwingMax', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Pivot',
       'Candle', 'CandleBody', '5EMA', '15EMA', '20EMA', '50EMA', '200EMA',
       'RSI', 'SuperTrend', 'Small_Swing_Min', 'Small_Swing_Max'],
      dtype='object')

In [131]:
upper_shadow =( (df["Candle"] == "Green") & (df["High"] - df["Close"] < df["Open"] - df["Low"]) || )
lower_shadow = df['Low'] - np.minimum(df['Open'], df['Close'])

df['Signal'] = np.where((upper_shadow > lower_shadow), 'Bearish',
                        np.where((lower_shadow > upper_shadow), 'Bullish', 'None'))


In [132]:
df[df['Signal'] != "None"][["Datetime","Signal"]]

,Datetime,Signal
515,2023-12-14 05:30:00,Bearish
516,2023-12-15 05:30:00,Bearish
517,2023-12-18 05:30:00,Bearish
518,2023-12-19 05:30:00,Bearish
519,2023-12-20 05:30:00,Bearish
...,...,...
610,2024-05-06 05:30:00,Bearish
611,2024-05-07 05:30:00,Bearish
612,2024-05-08 05:30:00,Bullish
613,2024-05-09 05:30:00,Bullish


In [130]:
df

,Day_Open,Day_High,Day_Low,Day_Close,Day_SuperTrend,Day_Candle,Day_CandleBody,Day_5EMA,Day_15EMA,Day_20EMA,...,5EMA,15EMA,20EMA,50EMA,200EMA,RSI,SuperTrend,Small_Swing_Min,Small_Swing_Max,Signal
515,21110.40,21210.90,21074.40,21182.7,1,Green,136.50,20991.287139,20622.160729,20471.126139,...,20991.287139,20622.160729,20471.126139,20016.998906,19165.818544,87.123324,1,18837.8,21210.9,None
516,21287.40,21492.30,21235.30,21456.7,1,Green,257.00,21146.424759,20726.478138,20564.990316,...,21146.424759,20726.478138,20564.990316,20073.457772,19188.613384,91.350033,1,18837.8,21492.3,None
517,21434.80,21482.80,21365.30,21418.7,1,Red,117.50,21237.183173,20813.005871,20646.296000,...,21237.183173,20813.005871,20646.296000,20126.212369,19210.803301,86.618276,1,18837.8,21492.3,None
518,21477.70,21505.10,21337.80,21453.1,1,Red,167.30,21309.155449,20893.017637,20723.134477,...,21309.155449,20893.017637,20723.134477,20178.247178,19233.114710,87.331141,1,18837.8,21505.1,None
519,21543.50,21593.00,21087.30,21150.2,1,Red,505.70,21256.170299,20925.165432,20763.807384,...,21256.170299,20925.165432,20763.807384,20216.362975,19252.190186,55.878274,1,18837.8,21593.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,22561.60,22588.80,22409.45,22442.7,1,Red,179.35,22505.820323,22455.911387,22429.344817,...,22505.820323,22455.911387,22429.344817,22238.022009,20955.902575,46.707719,1,21137.2,22794.7,None
611,22489.75,22499.05,22232.05,22302.5,1,Red,267.00,22438.046882,22436.734963,22417.264358,...,22438.046882,22436.734963,22417.264358,22240.550557,20969.301554,36.988077,1,21137.2,22794.7,None
612,22231.20,22368.65,22185.20,22302.5,1,Green,183.45,22392.864588,22419.955593,22406.334419,...,22392.864588,22419.955593,22406.334419,22242.979947,20982.567210,36.988077,1,21137.2,22794.7,None
613,22224.80,22307.75,21932.40,21957.5,1,Red,375.35,22247.743059,22362.148644,22363.588284,...,22247.743059,22362.148644,22363.588284,22231.784655,20992.268034,21.289492,1,21137.2,22794.7,None
